In [117]:
# imports
%reload_ext autoreload
%autoreload 2
%matplotlib inline 

import sys, os
from utils.basic_utils import *
from utils.pricing import *

pd.options.display.float_format = '{:,.2f}'.format

In [118]:
from utils.imports import *

In [119]:
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectFromModel
from sklearn.utils.validation import column_or_1d
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score, log_loss, precision_recall_fscore_support
from sklearn.metrics import precision_score, roc_auc_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

In [120]:
bench = '^GSPC'
sec_windows, stds = [5, 20, 60], 1
pred_fwd_windows = [60]
inv = incl_px = incl_name = False
y_col = 'fwdReturn'

### Get universe pricing

In [4]:
# TAKES ~8m on local drive, 3m on AWS for 1230 companies, do once and persist
excl_list = [] # ['BHF', 'ERI']
symbols_list = excl(config['companies'], excl_list)
%time px_close = get_mults_pricing(symbols_list)

Retrieving pricing: IDCC, (3774, 5)
Retrieving pricing: NTLA, (725, 5)
Retrieving pricing: SE, (357, 5)
Retrieving pricing: SXT, (3774, 5)
Retrieving pricing: KRC, (3774, 5)
Retrieving pricing: ADP, (3774, 5)
Retrieving pricing: AMN, (3774, 5)
Retrieving pricing: VMC, (3774, 5)
Retrieving pricing: AZUL, (491, 5)
Retrieving pricing: GWB, (2923, 5)
Retrieving pricing: JACK, (3774, 5)
Retrieving pricing: WEN, (3774, 5)
Retrieving pricing: ENSG, (2861, 5)
Retrieving pricing: FICO, (3774, 5)
Retrieving pricing: AMZN, (3774, 5)
Retrieving pricing: RSG, (3774, 5)
Retrieving pricing: CBD, (3774, 5)
Retrieving pricing: GT, (3774, 5)
Retrieving pricing: WY, (3774, 5)
Retrieving pricing: KEP, (3774, 5)
Retrieving pricing: DVA, (3774, 5)
Retrieving pricing: LLY, (3774, 5)
Retrieving pricing: SF, (3774, 5)
Retrieving pricing: HRL, (3774, 5)
Retrieving pricing: MLCO, (3085, 5)
Retrieving pricing: MNK, (1453, 5)
Retrieving pricing: DEI, (3123, 5)
Retrieving pricing: BRKR, (3774, 5)
Retrieving pricing

Retrieving pricing: ORAN, (3774, 5)
Retrieving pricing: JNPR, (3774, 5)
Retrieving pricing: DNKN, (1927, 5)
Retrieving pricing: STMP, (3774, 5)
Retrieving pricing: ADBE, (3774, 5)
Retrieving pricing: ATR, (3774, 5)
Retrieving pricing: TM, (3774, 5)
Retrieving pricing: HQY, (1170, 5)
Retrieving pricing: GRMN, (3774, 5)
Retrieving pricing: MDLZ, (3774, 5)
Retrieving pricing: B, (3774, 5)
Retrieving pricing: FIVN, (1251, 5)
Retrieving pricing: NTCT, (3774, 5)
Retrieving pricing: FMX, (3774, 5)
Retrieving pricing: ENR, (3774, 5)
Retrieving pricing: ABEV, (3774, 5)
Retrieving pricing: SMTC, (3774, 5)
Retrieving pricing: UMC, (3774, 5)
Retrieving pricing: KEX, (3774, 5)
Retrieving pricing: PBF, (1579, 5)
Retrieving pricing: TCF, (3774, 5)
Retrieving pricing: KEY, (3774, 5)
Retrieving pricing: TS, (3774, 5)
Retrieving pricing: EWBC, (3774, 5)
Retrieving pricing: GGG, (3774, 5)
Retrieving pricing: FFIN, (3774, 5)
Retrieving pricing: MSCI, (2857, 5)
Retrieving pricing: ZTO, (604, 5)
Retrieving 

Retrieving pricing: BDORY, (2324, 5)
Retrieving pricing: PACW, (3774, 5)
Retrieving pricing: WTR, (3774, 5)
Retrieving pricing: CMS, (3774, 5)
Retrieving pricing: NJR, (3774, 5)
Retrieving pricing: MKTX, (3619, 5)
Retrieving pricing: UTX, (3774, 5)
Retrieving pricing: HPT, (3774, 5)
Retrieving pricing: DNB, (3749, 5)
Retrieving pricing: TRV, (3774, 5)
Retrieving pricing: LNT, (3774, 5)
Retrieving pricing: RF, (3774, 5)
Retrieving pricing: SABR, (1242, 5)
Retrieving pricing: BWA, (3774, 5)
Retrieving pricing: SO, (3774, 5)
Retrieving pricing: FTNT, (2351, 5)
Retrieving pricing: AAN, (3774, 5)
Retrieving pricing: SWKS, (3774, 5)
Retrieving pricing: SCHW, (3774, 5)
Retrieving pricing: DXCM, (3510, 5)
Retrieving pricing: CCL, (3774, 5)
Retrieving pricing: CTRP, (3774, 5)
Retrieving pricing: CRH, (3774, 5)
Retrieving pricing: TCBI, (3774, 5)
Retrieving pricing: ADNT, (612, 5)
Retrieving pricing: IT, (3774, 5)
Retrieving pricing: VG, (3230, 5)
Retrieving pricing: GPS, (3774, 5)
Retrieving pr

Retrieving pricing: AVGO, (2424, 5)
Retrieving pricing: EPR, (3774, 5)
Retrieving pricing: ABBV, (1567, 5)
Retrieving pricing: KMI, (2041, 5)
Retrieving pricing: WBS, (3774, 5)
Retrieving pricing: MFG, (3113, 5)
Retrieving pricing: UA, (934, 5)
Retrieving pricing: WOR, (3774, 5)
Retrieving pricing: CIG, (3774, 5)
Retrieving pricing: AEIS, (3774, 5)
Retrieving pricing: CALM, (3774, 5)
Retrieving pricing: URGN, (475, 5)
Retrieving pricing: FL, (3774, 5)
Retrieving pricing: MBFI, (3773, 5)
Retrieving pricing: MIME, (840, 5)
Retrieving pricing: ACIW, (3774, 5)
Retrieving pricing: FELE, (3774, 5)
Retrieving pricing: ICE, (3359, 5)
Retrieving pricing: DDD, (3774, 5)
Retrieving pricing: NLSN, (2052, 5)
Retrieving pricing: SKX, (3774, 5)
Retrieving pricing: TOT, (3774, 5)
Retrieving pricing: LEG, (3774, 5)
Retrieving pricing: BRKS, (3774, 5)
Retrieving pricing: DDS, (3774, 5)
Retrieving pricing: WRI, (3774, 5)
Retrieving pricing: TTD, (623, 5)
Retrieving pricing: ELLI, (1997, 5)
Retrieving pri

Retrieving pricing: MCD, (3774, 5)
Retrieving pricing: TFX, (3774, 5)
Retrieving pricing: CME, (3774, 5)
Retrieving pricing: STE, (3774, 5)
Retrieving pricing: STZ, (3774, 5)
Retrieving pricing: NSRGY, (3774, 5)
Retrieving pricing: DPZ, (3701, 5)
Retrieving pricing: CMG, (3312, 5)
Retrieving pricing: NEU, (3774, 5)
Retrieving pricing: PNR, (3774, 5)
Retrieving pricing: PENN, (3774, 5)
Retrieving pricing: BMY, (3774, 5)
Retrieving pricing: SFNC, (3774, 5)
Retrieving pricing: CUZ, (3774, 5)
Retrieving pricing: VIPS, (1761, 5)
Retrieving pricing: RH, (1607, 5)
Retrieving pricing: GM, (2099, 5)
Retrieving pricing: IFF, (3774, 5)
Retrieving pricing: COO, (3774, 5)
Retrieving pricing: NTAP, (3774, 5)
Retrieving pricing: CS, (3774, 5)
Retrieving pricing: DUK, (3774, 5)
Retrieving pricing: KNX, (3774, 5)
Retrieving pricing: SIG, (3774, 5)
Retrieving pricing: BGNE, (790, 5)
Retrieving pricing: NGVT, (728, 5)
Retrieving pricing: BSBR, (2381, 5)
Retrieving pricing: PLD, (3774, 5)
Retrieving prici

Retrieving pricing: AIT, (3774, 5)
Retrieving pricing: WMT, (3774, 5)
Retrieving pricing: ALB, (3774, 5)
Retrieving pricing: AOS, (3774, 5)
Retrieving pricing: UGP, (3774, 5)
Retrieving pricing: ARI, (2390, 5)
Retrieving pricing: X, (3774, 5)
Retrieving pricing: SRE, (3774, 5)
Retrieving pricing: HR, (3774, 5)
Retrieving pricing: TME, (70, 5)
Retrieving pricing: CYBR, (1132, 5)
Retrieving pricing: CREE, (3774, 5)
Retrieving pricing: EVR, (3174, 5)
Retrieving pricing: WWD, (3774, 5)
Retrieving pricing: EBR, (2602, 5)
Retrieving pricing: GRFS, (1965, 5)
Retrieving pricing: TCEHY, (2320, 5)
Retrieving pricing: CSL, (3774, 5)
Retrieving pricing: PG, (3774, 5)
Retrieving pricing: TKC, (3774, 5)
Retrieving pricing: CCI, (3774, 5)
Retrieving pricing: RRC, (3774, 5)
Retrieving pricing: CPT, (3774, 5)
Retrieving pricing: MNRO, (3774, 5)
Retrieving pricing: MAN, (3774, 5)
Retrieving pricing: CABO, (953, 5)
Retrieving pricing: LANC, (3774, 5)
Retrieving pricing: INCY, (3774, 5)
Retrieving pricing

In [8]:
ticker = symbols_list[5]
print(ticker)

ADP


In [11]:
# save down to drive if refresh pricing
os.makedirs('tmp', exist_ok=True)
px_close.to_parquet('tmp/mult-co-px-ds')

In [121]:
px_close = pd.read_parquet('tmp/mult-co-px-ds')

In [122]:
px_close.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3916 entries, 2004-03-08 to 2019-03-25
Columns: 1230 entries, IDCC to LTM
dtypes: float64(1230)
memory usage: 36.8 MB


In [123]:
# use the latest saved data for profile and quote info
dates = read_dates('quote')
tgt_date = [dates[-1]] # last date saved in S3

quotes = load_csvs('quote_consol', tgt_date)
quotes.set_index('symbol', drop=False, inplace=True)

profile = load_csvs('summary_detail', ['assetProfile'])
profile.set_index('symbol', drop=False, inplace=True)

Loading file quote/csv/2019-03-25
Loading file summary-categories/assetProfile


In [124]:
profile.drop(profile[profile.symbol.isin(excl_list)].index, inplace=True)

all_equities = quotes[quotes.quoteType == 'EQUITY'].symbol.unique()
print('Delta quote: ', set(symbols_list) - set(all_equities))
# reduced subset, if any
sub_equities = set(px_close.columns.tolist()).intersection(all_equities)
print('Delta reduced set: ', set(symbols_list) - set(sub_equities))

eqty_symbols = profile[profile.symbol.isin(sub_equities)].symbol.unique().tolist()
delta_symb = set(symbols_list) - set(eqty_symbols)
print('Delta profile: ', len(delta_symb), delta_symb)

Delta quote:  {'KORS', 'PD', 'SHPG', 'SCG', 'FBR'}
Delta reduced set:  {'KORS', 'PD', 'SHPG', 'SCG', 'FBR'}
Delta profile:  29 {'CTAS', 'FBR', 'TAP', 'CIEN', 'DNB', 'FOXA', 'AMED', 'KORS', 'ROP', 'SHPG', 'ODFL', 'ESL', 'WSM', 'LRCX', 'PD', 'BSTI', 'SCG', 'CBOE', 'MFGP', 'JBT', 'REGN', '9984.T', 'VVC', 'CIG', 'NFX', 'MBFI', 'IBKR', 'DRE', 'ABMD'}


In [125]:
# Create a frame of market, sector and industry index (once)
# for relative performance calculations
sel_profiles = profile[profile.symbol.isin(all_equities)]
sel_profiles.groupby(['sector', 'industry'])[['industry']].count()
sectors = sel_profiles.sector.unique()
industries = sel_profiles.industry.unique()

f'Sectors: {sectors.shape[0]}, Industries: {industries.shape[0]}'

'Sectors: 11, Industries: 136'

In [126]:
%%time
indices_df = pd.concat([
    eq_wgt_indices(profile, px_close, 'sector', sectors, subset=eqty_symbols),
    eq_wgt_indices(profile, px_close, 'industry', industries, subset=eqty_symbols),
    to_index_form(get_symbol_pricing(bench)['close'], bench)
], axis=1)

CPU times: user 14.2 s, sys: 3.43 s, total: 17.6 s
Wall time: 9.16 s


### Dataset creation

In [127]:
df = get_symbol_pricing('AAPL')
ft_df = px_mom_feats(
    df['close'], ticker, stds, inv, incl_px, 
    sec_windows, incl_name)

In [128]:
ft_df[y_col] = px_fwd_rets(
        df.close, ticker, pred_fwd_windows).mean(axis=1)
    
co = px_mom_co_feats(
    df, indices_df, 
    [bench] + list(profile.loc[ticker, ['sector', 'industry']]))

In [129]:
co.dropna().shape

(3656, 16)

In [130]:
ft_df.dropna().shape

(3463, 11)

In [131]:
co.shape, ft_df.shape

((3916, 16), (3774, 11))

In [ ]:
px_close[ticker].drop_duplicates()

In [133]:
# Take a while ~40min to run on 1200 companies
# Can we make faster?
super_list = []
for ticker in tqdm(eqty_symbols):
    try:
        close = px_close[ticker].drop_duplicates()
        ft_df = px_mom_feats(
            close, ticker, stds, inv, incl_px, 
            sec_windows, incl_name)
        ft_df[y_col] = px_fwd_rets(
            close, ticker, pred_fwd_windows).mean(axis=1)
        df = get_symbol_pricing(ticker) #full retrieve
        co = px_mom_co_feats(
            df, indices_df, 
            [bench] + list(profile.loc[ticker, ['sector', 'industry']]))

        ft_df.loc[:, 'country'] = profile.loc[ticker,:].country
        ft_df.loc[:, 'currency'] = quotes.loc[ticker,:].currency

        ft_df = pd.concat([ft_df.dropna(), co.dropna()], axis=1)
        super_list.append(ft_df)
    except Exception as e:
        print("Exception: {0}\n{1}".format(ticker, e))
df_large = pd.concat(super_list, axis=0)
os.makedirs('tmp', exist_ok=True)
df_large.to_parquet('tmp/company-px_mom-large')
df_large.shape

 12%|█▏        | 140/1201 [00:44<04:57,  3.57it/s]Exception: BHF
Shape of passed values is (798, 29), indices imply (794, 29)
 31%|███       | 372/1201 [01:59<03:22,  4.10it/s]Exception: ERI
Shape of passed values is (2214, 29), indices imply (2210, 29)
100%|█████████▉| 1200/1201 [07:21<00:00,  3.45it/s]Exception: ZUO
Shape of passed values is (362, 29), indices imply (358, 29)
100%|██████████| 1201/1201 [07:22<00:00,  2.89it/s]


(3788726, 29)

In [134]:
df_large.shape

(3788726, 29)

In [112]:
# df_large.drop_duplicates().describe().T
# df_large.sort_index().groupby(by=df_large.index).count().mean().sort_values()
# df_large.dropna().describe().T

,count,mean,std,min,25%,50%,75%,max
PctChg1Stds,"78,436.00",0.01,0.77,-14.00,0.00,0.00,0.00,21.00
PctMA50,"78,436.00",1.02,0.11,0.09,0.97,1.02,1.07,2.66
PctMA200,"78,436.00",1.07,0.20,0.02,0.97,1.08,1.18,2.58
RollVol30,"78,436.00",0.38,0.27,0.04,0.22,0.31,0.44,3.73
PctChg5,"78,436.00",0.00,0.06,-0.88,-0.02,0.00,0.03,2.97
PctChg20,"78,436.00",0.02,0.13,-0.90,-0.04,0.02,0.07,4.43
PctChg60,"78,436.00",0.05,0.21,-0.95,-0.05,0.05,0.15,3.93
Pct52WkH,"78,436.00",0.86,0.17,0.01,0.80,0.91,0.98,1.00
Pct52WkL,"78,436.00",1.59,0.70,1.00,1.23,1.40,1.71,12.07
fwdReturn,"78,436.00",0.05,0.21,-0.95,-0.06,0.05,0.14,3.61
